<h2>Desarrollo del Desafio 2 - Machine Learning


<h3>Grupo 7

<li>Laura Jazmín Chao
<li>Juan Nicolás Capistrano
<li>Betiana Srur
<li>Ignacio Mendieta
<li>Marecelo Carrizo

**Desafio 2** : Aplicar las tecnicas de Machine Learning para calcular el precio por metro cuadrado de las propiedades.

In [2]:
import numpy as np
import pandas as pd

# Libreria Grafica.
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

# Libreria para analisis ML
from sklearn import datasets, linear_model

In [3]:
# Definimos la ruta de la información.
data_propiedades = "Data/properati.csv"

# Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

# Chequeamos que los datos se hayan importado correctamente
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
94905,94905,sell,apartment,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,3435874.0,NaN,NaN,...,133.0,2611.111111,3533.834586,NaN,NaN,7300.0,http://www.properati.com.ar/1bgbx_venta_depart...,- Excelente estado - Balcón terraza al frente ...,Excelente departamento de 4 ambientes con 3 co...,https://thumbs4.properati.com/6/59F3suDdu_kQ6a...
106342,106342,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,...,37.0,NaN,36906.648649,NaN,NaN,NaN,http://www.properati.com.ar/1bw7d_venta_depart...,Departamentos de primera calidad. Ideal invers...,Departamento - Echesortu,NaN
35369,35369,sell,house,Monte Grande,|Argentina|Bs.As. G.B.A. Zona Sur|Esteban Eche...,Argentina,Bs.As. G.B.A. Zona Sur,NaN,"-34.8423335,-58.4546261",-34.842334,...,150.0,705.882353,800.000000,NaN,NaN,NaN,http://www.properati.com.ar/18e6t_venta_casa_m...,CODIGO: ubicado en: Juarez 2823 - Publicado ...,Hermosa casa construida sobre dos lotes.,https://thumbs4.properati.com/5/KRjJghovF_mYRf...
15275,15275,sell,house,Morón,|Argentina|Bs.As. G.B.A. Zona Oeste|Morón|,Argentina,Bs.As. G.B.A. Zona Oeste,3430544.0,"-34.6904966,-58.6300651",-34.690497,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/16uo7_venta_casa_m...,CODIGO: 690-v-15050 ubicado en: Mosquera 1872...,MOSQUERA - MORON SUR,https://thumbs4.properati.com/4/dOEBkJSIzkVc3p...
98547,98547,sell,house,Nordelta,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre|Nord...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1bln9_venta_casa_n...,Lindísima casa 5 amb desarrollada en dos plant...,Casa - Los Castores,https://thumbs4.properati.com/0/STcrcD0ZJ9So11...


In [4]:
# Chequeamos cantidad de registros y cantidad de columnas
data.shape

(121220, 26)

<h2>Limpieza de Datos.

In [5]:
# Creamos una mask para traer los valores en monedas que no son USD ni ARS.
currency_OTHER_CURRENCY_mask = (data.currency == 'PEN') | (data.currency == 'UYU')
# Observamos esos registros
data.loc[currency_OTHER_CURRENCY_mask, :]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
50387,50387,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e8z_venta_depart...,Ref.: 55:PREVENTA ::: OPORTUNIDAD ::: Dúplex a...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/9/N_Qr_5X_vOq--b...
50388,50388,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/19e90_venta_depart...,Ref.: 56Duplex en venta de 1 dormitorio con ap...,DUPLEX A LA VENTA | Liniers 1200 | Tandil,https://thumbs4.properati.com/8/_cBXAyr_9pDySg...
107390,107390,sell,house,San Roque,|Argentina|Mendoza|San Roque|,Argentina,Mendoza,3836586.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1by0s_venta_casa_s...,"Casa con Calefacción central, Mendoza, San Roq...",Casa - San Vicente,https://thumbs4.properati.com/2/pA0RbuDKRK6GgV...


In [6]:
# Hacemos drop de esos registros y chequeamos la diferencia entre cantidad original y actual
cant_registros = data.shape[0]
data.drop(data.loc[currency_OTHER_CURRENCY_mask, :].index, inplace = True)
print(f'Cantidad de registros original - registro actuales: {cant_registros - data.shape[0]}')

Cantidad de registros original - registro actuales: 3


In [10]:
# Creamos la lista de columnas a filtrar
drop_columns = ['properati_url', 'Unnamed: 0', 'operation','image_thumbnail','expenses','lat-lon','geonames_id','lat', 'lon']

# Creamos una variable nueva para guardar los datos (el cambio se realiza inplace=False)
data_filtered = data.drop(drop_columns, axis=1)
data_filtered.head(5)

,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado
4,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...


**Esto no es necesario ahora ¿?**

In [11]:
# Creamos un diccionario con la nueva agrupación
provincia={'Capital Federal':'CABA','Bs.As. G.B.A. Zona Norte': 'Bs. As. G.B.A.', 'Bs.As. G.B.A. Zona Sur':'Bs. As. G.B.A.', 'Córdoba':'Córdoba','Santa Fe':'Santa Fe','Buenos Aires Costa Atlántica':'Buenos Aires', 'Bs.As. G.B.A. Zona Oeste':'Bs. As. G.B.A.', 'Buenos Aires Interior':'Buenos Aires','Río Negro':'Resto del país','Neuquén':'Resto del país', 'Mendoza':'Resto del país', 'Tucumán':'Resto del país', 'Corrientes':'Resto del país', 'Misiones':'Resto del país', 'Entre Ríos':'Resto del país', 'Salta':'Resto del país', 'Chubut':'Resto del país', 'San Luis':'Resto del país', 'La Pampa':'Resto del país','Formosa':'Resto del país','Chaco': 'Resto del país', 'San Juan':'Resto del país','Tierra Del Fuego':'Resto del país', 'Catamarca':'Resto del país', 'Jujuy': 'Resto del país', 'Santa Cruz':'Resto del país', 'La Rioja':'Resto del país', 'Santiago Del Estero':'Resto del país'}

In [14]:
# Mapeamos las provincias con el diccionario y ponermos el grupo en una columna nueva
data_filtered['province_group'] = data_filtered.state_name.map(provincia)
display(data_filtered.sample(5))


,property_type,place_name,place_with_parent_names,country_name,state_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,description,title,province_group
56397,apartment,Ramos Mejía,|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza...,Argentina,Bs.As. G.B.A. Zona Oeste,55000.0,USD,970447.50,55000.00,NaN,1.0,NaN,55000.000000,NaN,NaN,"DEPARTAMENTO MONOAMBIENTE, VICENTE LOPEZ 295, ...",DEPARTAMENTO EN VENTA,Bs. As. G.B.A.
87299,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,950000.0,ARS,940089.96,53279.49,NaN,40.0,NaN,23750.000000,NaN,NaN,"Unidad: ""15"" con vista al Centro de la Ciudad,...",Departamento - Alto Alberdi,Córdoba
86061,apartment,San Telmo,|Argentina|Capital Federal|San Telmo|,Argentina,Capital Federal,76000.0,USD,1340982.00,76000.00,32.0,28.0,2375.000000,2714.285714,NaN,NaN,Departamento en venta de 1 ambiente en el barr...,Monoambiente al frente apto profesional - vent...,CABA
112610,apartment,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,NaN,NaN,NaN,NaN,46.0,42.0,NaN,NaN,NaN,NaN,Te invitamos a conocer:Windows TowerLa vanguar...,Departamento - Nueva Cordoba,Córdoba
103460,apartment,Villa Rosa,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|Vill...,Argentina,Bs.As. G.B.A. Zona Norte,300000.0,USD,5293350.00,300000.00,170.0,72.0,1764.705882,4166.666667,NaN,NaN,Winterra S.A.Catamarca 1150CMCPSI5015,Lagoon Pilar 3 ambientes con terraza y Vista a...,Bs. As. G.B.A.


Córdoba                9254
Rosario                8504
Mar del Plata          6534
Tigre                  3324
Nordelta               3315
                       ... 
Bouwer                    1
Golf Club Argentino       1
Villa Argüello            1
Calmayo                   1
Barrio Melazzi            1
Name: place_name, Length: 1060, dtype: int64

<h2>Tomamos solamente CABA para el analisis.

In [31]:
# Creamos una máscara y la aplicamos al dataframe anterior para traer los registros que necesitamos
data_caba_mask = data_filtered.state_name == 'Capital Federal' 
data_caba = data_filtered.loc[data_caba_mask, :]

# Dropeamos la columna creada anteriormente.
data_caba.drop(columns=['province_group', 'country_name', 'state_name', 'place_with_parent_names','description', 'title'], inplace=True)

# Chequeamos cómo quedaron los datos
data_caba.head(3)

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Vemos la cantidad de datos solamente de CABA.
data_caba.shape

(32314, 13)

In [32]:
# Revisamos si hay barrios de caba mal indentificados "Ejemplo: Capital Federal"
pd.set_option('display.max_rows', 1000) # Para mostrar todas las filas.


In [36]:
# Dropeamos los de Capital Federal
capi_mask = data_caba.place_name == 'Capital Federal'
data_caba.drop(data_caba.loc[capi_mask, :].index, inplace=True)

In [39]:
data_caba.shape

(31019, 12)

In [40]:
data_caba.head()

,property_type,place_name,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms
0,PH,Mataderos,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN
2,apartment,Mataderos,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN
3,PH,Liniers,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN
7,apartment,Belgrano,138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN
8,apartment,Belgrano,195000.0,USD,3440677.5,195000.0,65.0,60.0,3000.000000,3250.000000,NaN,NaN
